In [2]:
'''
CAP6615 - Spring 2022
Assignment #1 - Single- and Multi-Layer Shallow Neural Networks
'''

import numpy as np
import os
import sys
from PIL import Image
import pandas

class performanceMetrics(object):
    def __init__(self, fh, ffa):
        self.fh = fh
        self.ffa = ffa

class Perceptron():
    def __init__(self):
        '''
        Initialize perceptron model.
        '''

    def activation_fn(self,x):
        #     Sigmoid Activation func: f(x) = 1/(1+e^(-x))
        return 1/(1+np.exp(-x))
        
    def _initialize_weights(self):
        '''
        Initialize the weights, initalize using random numbers.
        '''
        self.weights = np.random.randn()

    def initialize_all_weights_to_zeros(self):
        '''
        Initialize the weights, initalize using random numbers.
        '''

    def predict(self,inputVector, weightMatrix):
        '''
        Make a prediction on an array of inputs
        '''
        predictionVector = []
        for i in range(len(weightMatrix)):
            row_sum = 0
            for j in range(len(weightMatrix[0])-1):
                row_sum += (inputVector[j]*weightMatrix[j][i])
            predictionVector.append(self.activation_fn(row_sum))
        # print('Prediction')
        # print(predictionVector)   
        return predictionVector

    def calculate_performance_metrics(self, predictionVector, inputVector, outputVector):
        '''
        Calculates percent error.
        '''
        totalBlackPixelCount = sum(x == 0 for x in inputVector)
        totalWhitePixelCount = sum(x == 1 for x in inputVector)
        wrongBlackPixelCount = 0
        rightBlackPixelCount = 0
        for i in range(len(outputVector)):
            if outputVector[i] < 0.0001:
                if  abs(predictionVector[i] - inputVector[i]) < 0.0001:
                    rightBlackPixelCount += 1
                else:
                    wrongBlackPixelCount += 1
        fh = rightBlackPixelCount/totalBlackPixelCount
        ffa = wrongBlackPixelCount/totalWhitePixelCount         
        metric = performanceMetrics(fh,ffa)
        return metric    

    def updateBias(self, predictionVector, outputVector, inputVector, biasVector):
        '''
        Trains the model on data X with desired outputs d.
        '''
        learningRate = 0.8
        for i in range(len(outputVector)):
            error = outputVector[i] - predictionVector[i]
            biasVector[i][0] += (learningRate*inputVector[i]*error) 
        return biasVector


    def fit(self,predictionVector,outputVector,inputVector,weightMatrix):
        '''
        Trains the model on data X with desired outputs d.
        '''
        learningRate = 0.8
        for i in range(len(weightMatrix)):
            error = outputVector[i] - predictionVector[i]
            for j in range(len(weightMatrix[0])):
                weightMatrix[j][i] += (learningRate*inputVector[j]*error) 
        return weightMatrix

    def main(self):
        tableObject = {}
        for i in range(10):
            tableObject[str(i)+'_fh'] = []
            # tableObject[str(i)+'_ffa'] = []
        
        for i in range(10):
            path = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'numbers','pixil-frame-'+ str(i) +'.png')
            im = Image.open(path, 'r')
            gray = im.convert('L')
            bw = gray.point(lambda x: 0 if x<135 else 1, '1')
            pix_val_1 = list(bw.getdata())
            weightMatrix = np.random.rand(len(pix_val_1),len(pix_val_1))
            # biasVector = np.random.rand(len(pix_val_1),1)
            epochs = 4
            for j in range(epochs):
                prediction = self.predict(pix_val_1,weightMatrix)
                # trainMetric = self.calculate_performance_metrics(prediction,pix_val_1,pix_val_1)
                # print('Fraction of Hits')
                # print(trainMetric.fh)    
                # print('Fraction of False Alarms')
                # print(trainMetric.ffa)
                weightMatrix = self.fit(prediction,pix_val_1,pix_val_1,weightMatrix)
                # biasVector = self.updateBias(prediction,pix_val_1,pix_val_1,biasVector)
            # print('Input')
            # print(pix_val_1)         
            # print('Difference')
            # print (sum(np.array(prediction) - np.array(pix_val_1)))        
 
            for k in range(10):
                testPath = os.path.join(os.path.dirname(os.path.abspath(sys.argv[1])),'numbers','pixil-frame-' + str(k) + '.png')
                testIm = Image.open(testPath, 'r')
                testGray = testIm.convert('L')
                testBW = testGray.point(lambda x: 0 if x<135 else 1, '1')
                testInput = list(testBW.getdata())
                testPrediction = self.predict(testInput,weightMatrix)
                testMetric = self.calculate_performance_metrics(testPrediction,testInput,testInput)
                tableObject[str(k)+'_fh'].append(round(testMetric.fh,2))
                # tableObject[str(k)+'_ffa'].append(round(testMetric.ffa,2))

           
        print(pandas.DataFrame(tableObject))
        pandas.DataFrame(tableObject).to_csv('Test_Results.csv')
        
if __name__ == "__main__":
    perceptron = Perceptron()
    perceptron.main()

   0_fh  1_fh  2_fh  3_fh  4_fh  5_fh  6_fh  7_fh  8_fh  9_fh
0  1.00  0.63  0.81  0.88  0.69  0.82  0.84  0.65  0.84  0.84
1  0.46  1.00  0.48  0.52  0.30  0.50  0.47  0.41  0.47  0.47
2  0.82  0.67  1.00  0.94  0.65  0.86  0.88  0.74  0.89  0.88
3  0.78  0.63  0.82  1.00  0.57  0.83  0.79  0.74  0.82  0.84
4  0.51  0.31  0.48  0.48  1.00  0.49  0.50  0.43  0.50  0.53
5  0.82  0.69  0.85  0.94  0.65  1.00  0.93  0.70  0.89  0.93
6  0.89  0.69  0.92  0.94  0.70  0.99  1.00  0.67  0.95  0.95
7  0.42  0.37  0.47  0.53  0.37  0.44  0.41  1.00  0.45  0.45
8  0.89  0.69  0.93  0.97  0.70  0.94  0.95  0.74  1.00  0.95
9  0.89  0.69  0.92  1.00  0.74  0.99  0.95  0.74  0.95  1.00
